# Final Project for DTSA-5510

Kevin Juandi <br> 2023-11-25<br> https://github.com/Chiuchiyin/

## Introduction and Problem Description

Welcome to my final project for DTSA-5510! Before running this notebook, please ensure you have all required dependancies installed in your build. All required packages are listed in the next section. If you want to run the code, some parts of the code might take very long time to run. At the time of this writing, I have not attempted to train a model but I expect it to take a couple of hours. Please be patient with it. Thank you for reviewing my project!

For the previous module **DTSA-5509**, I worked on [Heartbeat ECG Classification](https://github.com/Chiuchiyin/Heartbeat-ECG-Classification). I would like to expand my repetoire in health analytics and have decided to work on Pneumonia Classification.

The data I will use can be found at Mendeley Data[<sup>[1]</sup>](https://data.mendeley.com/datasets/rscbjbr9sj/3). The data contain images of both chest X-Rays and Optical Coherence Tomography. The original paper uses deep learning algorithm which I would not attempt to replicate due to computational requirements. I will just use the chest X-rays in this project. I have included the data in the Github Repository so if you decided to download the data so you don't have to download the entire dataset and extract them yourself.

The chest X-ray data in "train" contains 1.18 GB of data - consisting of 5233 "normal" chest X-rays, 1345 X-rays with viral pneumonia, and 2538 X-rays with bacterial pneumonia. The "test" data contains 105 MB of data - there are 235 "normal" chest X-rays, 626 with viral pneumonia, and 242 with bacterial pneumonia.

## Loading dependancies

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import cv2 as cv
import matplotlib.pyplot as plt
import os
import math
from sklearn.decomposition import PCA, NMF
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
import time
import itertools
import random

## Helper functions
We would use these functions to reduce clutter

In [10]:
# Function to read a file into numpy.
def readXRay(path, pixDesired = 1000, n_imgs = "use all"):
    """
        input: string path, of form ("./data/ os.listdir(parent directory)
        input: int pixDesired, the number of pixels pixDesired X pixDesired in which to read the data. If None, no transformation is performed.
        input: int n_imgs, number of images to be read from file. Preset is string "use all", which uses all the images in the provided path.
        output: np.ndarray data
    """
    
    # Initialize data
    firstImagePath = path + os.listdir(path)[0]
    firstPic = cv.imread(firstImagePath, 0)
    if pixDesired != None:
        data = cv.resize(firstPic, (pixDesired, pixDesired))
        data = data.reshape(1, pixDesired ** 2)# If performance allows, increase resize!
    else:
        data = data.reshape(1, firstPic.shape[0] * firstPic.shape[1])
    
    # Populate data
    
    if n_imgs == "use all":
        for img in os.listdir(path)[1:]:
            filepath = path + img
            pic = cv.imread(filepath, 0)
            if pixDesired != None:
                pic = cv.resize(pic, (pixDesired, pixDesired))
                pic = pic.reshape(1, pixDesired ** 2)
            else:
                pic = pic.reshape(1, pic.shape[0] * pic.shape[1])
            
            data = np.append(data, pic, axis = 0)
            
    else:
        for img in os.listdir(path)[1:n_imgs]:
            filepath = path + img
            pic = cv.imread(filepath, 0)
            if pixDesired != None:
                pic = cv.resize(pic, (pixDesired, pixDesired)) # If performance allows, increase resize!
                pic = pic.reshape(1, pixDesired ** 2)
            else:
                pic = pic.reshape(1, pic.shape[0] * pic.shape[1])
            
            data = np.append(data, pic, axis = 0)
        
    return(data)

## Load the Data

In [14]:
%cd "./chest_xray/"

os.listdir()

C:\Users\silve\OneDrive\Dokumente\Github\DTSA5510-Unsupervised-Learning-Final-Project\chest_xray


['test', 'train']

In [16]:
# Example XRay. Citation https://www.geeksforgeeks.org/python-opencv-cv2-imshow-method/
# Set file path
startTime = time.time()
repopath = "./train/" + "NORMAL/"
filepath = repopath + os.listdir(repopath)[0]

#Read in test image
testpic = cv.imread(filepath, 0)
#testpic = cv.resize(testpic, (256, 256))

endTime = time.time()

##Show test image
#testWindow = "Example XRAY. This is a healthy lung."
#cv.imshow(testWindow, testpic)
#cv.waitKey(0)  #Wait for the user to press a key before closing the window
#cv.destroyAllWindows() 

print(testpic.shape)
print(endTime - startTime)

(2476, 2572)
0.038080692291259766


In [19]:
# Set up paths of repositories to read data from
paths = ["./train/NORMAL/", "./train/VIRAL/", "./train/BACTERIAL/", 
         "./test/NORMAL/", "./test/VIRAL/", "./test/BACTERIAL/"]

#Initialize data
firststartTime = time.time()
data = readXRay(path = paths[0], pixDesired = 100, n_imgs = "use all")
firstendTime = time.time()
print(firstendTime - firststartTime) #How long did this repo take to convert?
print(data.shape) #How many files in this repo?

#Populate data with the other repositories
for path in paths[1:]:
    startTime = time.time()
    this_data = readXRay(path = path, pixDesired = 100, n_imgs = "use all")
    endTime = time.time()

    print(endTime - startTime) #How long did this repo take to convert?
    print(this_data.shape) #How many files in this repo?
    data = np.append(data, this_data, axis = 0)

print(endTime - firststartTime)
print(data.shape)

17.501272439956665
(1349, 10000)
6.932614803314209
(1345, 10000)
16.23611283302307
(2538, 10000)
2.336109161376953
(234, 10000)
1.498124122619629
(148, 10000)
2.1744823455810547
(242, 10000)
46.7120566368103
(5856, 10000)


In [20]:
# Records 3 class labels in np.ndarray
Normal1 = np.asarray(["NORMAL" for i in range(1349)])
Viral1 = np.asarray(["VIRAL" for i in range(1345)])
Bacterial1 = np.asarray(["BACTERIAL" for i in range(2538)])
Normal2 = np.asarray(["NORMAL" for i in range(234)])
Viral2 = np.asarray(["VIRAL" for i in range(148)])
Bacterial2 = np.asarray(["BACTERIAL" for i in range(242)])
labs = np.append(Normal1, Viral1, axis = 0)
labs = np.append(labs, Bacterial1, axis = 0)
labs = np.append(labs, Normal2, axis = 0)
labs = np.append(labs, Viral2, axis = 0)
labs = np.append(labs, Bacterial2, axis = 0)

assert labs.shape[0] == data.shape[0]

print(labs.shape)

(5856,)


In [21]:
# Records 2 class labels in np.ndarray
Infected1 = np.asarray(["INFECTED" for i in range(1345 + 2538)])
Infected2 = np.asarray(["INFECTED" for i in range(148 + 242)])

labs2 = np.append(Normal1, Infected1, axis = 0)
labs2 = np.append(labs2, Normal2, axis = 0)
labs2 = np.append(labs2, Infected2, axis = 0)

assert labs2.shape[0] == data.shape[0]

print(labs2.shape)

(5856,)
